# LLMS-Generator Experiments

For experimentation with the LLMS-Generator Agent.

## Setup

To run this notebook, ensure you've authenticated to Google Cloud and installed the project's dependencies.

To setup the Google Cloud environment:

```bash
source scripts/setup-env.sh
```

Then to install the package dependencies into the virtual environment, use the `uv` tool:

1. From your agent's root directory, run `make install` to set up the virtual environment (`.venv`).
2. In this Jupyter notebook, select the kernel from the `.venv` folder to ensure all dependencies are available.

### Import Libraries

In [1]:
import os

import vertexai
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.auth import default
from google.genai.types import Content, Part
from IPython.display import Markdown, display

### Load Environment Variables

In [2]:
dotenv_path = os.path.abspath('../.env')

if os.path.exists(dotenv_path):
    print(f"Loading environment variables from: {dotenv_path}")
    load_dotenv(dotenv_path=dotenv_path)
else:
    print(f"Warning: .env file not found at {dotenv_path}")

staging_project_id = os.getenv("GOOGLE_CLOUD_STAGING_PROJECT")
if staging_project_id:
    os.environ["GOOGLE_CLOUD_PROJECT"] = staging_project_id
    print(f"Set GOOGLE_CLOUD_PROJECT environment variable to: {staging_project_id}")


Loading environment variables from: /home/darren/localdev/Python/llms-gen/.env
Set GOOGLE_CLOUD_PROJECT environment variable to: scratch-dev-428715


### Ensure We're Authenticating to the Right Project

In [3]:
credentials, project_id = default()  # To use ADC
vertexai.init(project=staging_project_id, location="europe-west4", credentials=credentials) # type: ignore

## Running the Agent

In [4]:
from llms_gen_agent.agent import root_agent

16:31:34.873:llms_gen_agent - INFO: Logger initialised for llms_gen_agent.
16:31:34.873:llms_gen_agent - DEBUG: DEBUG level logging enabled.
16:31:35.451:llms_gen_agent - DEBUG: agent_name set to llms_gen_agent
16:31:35.452:llms_gen_agent - DEBUG: project_id set to scratch-dev-428715
16:31:35.452:llms_gen_agent - DEBUG: location set to global
16:31:35.452:llms_gen_agent - DEBUG: model set to gemini-2.5-flash
16:31:35.453:llms_gen_agent - DEBUG: genai_use_vertexai set to True


In [5]:
# Session and Runner
APP_NAME = "generate_llms_client"
USER_ID="test_user"
SESSION_ID="test_session"
TEST_REPO_PATH = "/home/darren/localdev/gcp/adk-docs" # You can change this to any repository you want to test

async def setup_session_and_runner():
    session_service = InMemorySessionService()
    session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)
    return session, runner

In [6]:
# Agent Interaction
async def call_agent_async(query):
    content = Content(role='user', parts=[Part(text=query)])
    _, runner = await setup_session_and_runner()
    events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    async for event in events:
        if function_calls := event.get_function_calls():
            tool_name = function_calls[0].name
            display(Markdown(f"_Using tool {tool_name}..._"))
        elif event.actions and event.actions.transfer_to_agent:
            personality_name = event.actions.transfer_to_agent
            display(Markdown(f"_Delegating to agent: {personality_name}..._"))
        elif event.is_final_response() and event.content and event.content.parts:
            final_message = event.content.parts[0].text
            display(Markdown("### Final Message"))
            display(Markdown(final_message))
        else:
            print(f"event: {event}")


In [7]:
query = f"Generate the llms.txt file for the repository at {TEST_REPO_PATH}"
await call_agent_async(query)

_Using tool discover_files..._

16:31:36.716:llms_gen_agent - DEBUG: Entering discover_files with repo_path: /home/darren/localdev/gcp/adk-docs
16:31:36.718:llms_gen_agent - DEBUG: /home/darren/localdev/gcp/adk-docs/CONTRIBUTING.md
/home/darren/localdev/gcp/adk-docs/README.md
/home/darren/localdev/gcp/adk-docs/docs/contributing-guide.md
/home/darren/localdev/gcp/adk-docs/docs/community.md
/home/darren/localdev/gcp/adk-docs/docs/index.md
/home/darren/localdev/gcp/adk-docs/docs/deploy/gke.md
/home/darren/localdev/gcp/adk-docs/docs/deploy/cloud-run.md
/home/darren/localdev/gcp/adk-docs/docs/deploy/agent-engine.md
/home/darren/localdev/gcp/adk-docs/docs/deploy/index.md
/home/darren/localdev/gcp/adk-docs/docs/sessions/state.md
/home/darren/localdev/gcp/adk-docs/docs/sessions/session.md
/home/darren/localdev/gcp/adk-docs/docs/sessions/memory.md
/home/darren/localdev/gcp/adk-docs/docs/sessions/index.md
/home/darren/localdev/gcp/adk-docs/docs/sessions/express-mode.md
/home/darren/localdev/gcp/adk-docs/docs/tutorials/agent-te

event: content=Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        id='adk-2a867e5a-e02b-495e-a4ef-a53037ea862a',
        name='discover_files',
        response={
          'files': [
            '/home/darren/localdev/gcp/adk-docs/CONTRIBUTING.md',
            '/home/darren/localdev/gcp/adk-docs/README.md',
            '/home/darren/localdev/gcp/adk-docs/docs/contributing-guide.md',
            '/home/darren/localdev/gcp/adk-docs/docs/community.md',
            '/home/darren/localdev/gcp/adk-docs/docs/index.md',
            <... 80 more items ...>,
          ],
          'status': 'success'
        }
      )
    ),
  ],
  role='user'
) grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None invocation_id='e-f5c3bb00-2a44-4052-8f5d-c1a6c06a8c43' author='generate_

_Using tool document_summariser_agent..._

event: content=Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        id='adk-1a6f05ca-5e58-44b1-8fb1-09a85ce107e8',
        name='document_summariser_agent',
        response={
          'summaries': [
            {
              <... Max depth ...>: <... Max depth ...>,
              <... Max depth ...>: <... Max depth ...>
            },
            {
              <... Max depth ...>: <... Max depth ...>,
              <... Max depth ...>: <... Max depth ...>
            },
            {
              <... Max depth ...>: <... Max depth ...>,
              <... Max depth ...>: <... Max depth ...>
            },
            {
              <... Max depth ...>: <... Max depth ...>,
              <... Max depth ...>: <... Max depth ...>
            },
            {
              <... Max depth ...>: <... Max depth ...>,
              <... Max depth ...>: <... Max depth ...>
            },
            <... 5 more items ...>,
          ]
        }
      )
    ),


### Final Message

Here are the summaries of the files in the repository:

| File Path | Summary |
|---|---|
| /home/darren/localdev/gcp/adk-docs/CONTRIBUTING.md | This document outlines the process for contributing to the project, emphasizing the need to sign a Contributor License Agreement and adhere to Google's Open Source Community Guidelines. It details the contribution workflow, including finding tasks, setting up the development environment, and the code review process via GitHub Pull Requests. The guide ensures a structured approach for new contributors to engage with the project. |
| /home/darren/localdev/gcp/adk-docs/README.md | This file introduces the Agent Development Kit (ADK), an open-source, code-first toolkit designed for building, evaluating, and deploying AI agents with flexibility and control. It highlights key features such as a rich tool ecosystem, modular multi-agent systems, built-in observability, and deployment flexibility across various platforms. The document also provides installation instructions for Python and Java, links to comprehensive documentation, and encourages community contributions. |
| /home/darren/localdev/gcp/adk-docs/docs/contributing-guide.md | This guide details how to contribute to the Agent Development Kit (ADK), covering both its core framework (Python and Java) and documentation. It specifies different GitHub repositories for each component and mandates signing a Contributor License Agreement and reviewing community guidelines before contributing. The document outlines various contribution methods, including reporting issues, suggesting enhancements, improving documentation, and writing code, all managed through GitHub Pull Requests and code reviews. |
| /home/darren/localdev/gcp/adk-docs/docs/community.md | This document provides a list of community resources for the Agent Development Kit (ADK), including community-provided translations of the documentation in Chinese, Korean, and Japanese. It also features a collection of tutorials, guides, and blog posts that cover ADK features, use cases, and integrations, alongside several videos and screencasts showcasing ADK functionalities. The page encourages community members to contribute their own resources and provides guidance on how to do so. |
| /home/darren/localdev/gcp/adk-docs/docs/index.md | This document serves as the main entry point for the Agent Development Kit (ADK) documentation, defining ADK as a flexible and modular framework for developing and deploying AI agents. It emphasizes ADK's model-agnostic and deployment-agnostic nature, providing quick installation instructions for Python and Java, and links to essential resources like quickstarts, tutorials, and API references. The page also highlights core features such as flexible orchestration, multi-agent architecture, a rich tool ecosystem, deployment readiness, and built-in evaluation and safety features. |
| /home/darren/localdev/gcp/adk-docs/docs/deploy/gke.md | This document offers a detailed guide for deploying an Agent Development Kit (ADK) agent to Google Kubernetes Engine (GKE), covering both manual and automated deployment methods. It provides instructions for setting up environment variables, enabling necessary APIs, granting permissions, and creating agent code, Dockerfiles, and Kubernetes manifest files. The guide also explains how to build container images, configure Kubernetes service accounts for Vertex AI, verify deployments, test the agent via UI or API, and troubleshoot common issues. |
| /home/darren/localdev/gcp/adk-docs/docs/deploy/cloud-run.md | This document provides instructions for deploying an Agent Development Kit (ADK) agent to Google Cloud Run, offering deployment via the `adk deploy cloud_run` CLI command or the `gcloud run deploy` command. It outlines agent configurations for Python and Java, details environment variable setup, and explains the deployment payload. The guide includes comprehensive steps for project structure, code files, and deployment commands, along with methods for testing the deployed agent through UI or API, and managing authenticated access. |
| /home/darren/localdev/gcp/adk-docs/docs/deploy/agent-engine.md | This document outlines the process for deploying an Agent Development Kit (ADK) agent to Vertex AI Agent Engine, a fully managed Google Cloud service designed for scaling AI agents in production, specifically for Python. It details prerequisites like a Google Cloud Project with Vertex AI API enabled, authenticated `gcloud CLI`, and a GCS bucket for staging. The guide covers defining and preparing an agent for deployment using `AdkApp`, initializing the Vertex AI SDK, local testing, and deploying via ADK CLI or Python SDK, concluding with instructions for interacting with and cleaning up deployed agents. |
| /home/darren/localdev/gcp/adk-docs/docs/deploy/index.md | This document provides an overview of various deployment options for Agent Development Kit (ADK) agents, enabling their transition from local development to scalable production environments. It highlights three primary Google Cloud deployment platforms: Agent Engine in Vertex AI for managed auto-scaling, Cloud Run for container-based applications, and Google Kubernetes Engine (GKE) for enhanced control and open model support. Additionally, it suggests deploying agents to other container-friendly infrastructures like Docker or Podman by manually packaging them into container images. |
| /home/darren/localdev/gcp/adk-docs/docs/sessions/state.md | This document explains the `state` attribute within an ADK `Session`, which acts as an agent's scratchpad for dynamic information during conversations. It clarifies that `session.state` stores serializable key-value pairs to personalize interactions, track progress, and accumulate data, while detailing how prefixes (`user:`, `app:`, `temp:`) define scope and persistence. The guide emphasizes recommended methods for state updates via `output_key`, `EventActions.state_delta`, or `CallbackContext`/`ToolContext`, and strongly advises against direct modification of `session.state` outside of managed contexts to ensure data integrity and persistence. |